In [1]:
import bs4 
import requests
import pandas as pd
import numpy as np
import os
import requests
import re

import warnings
warnings.filterwarnings("ignore")


In [2]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [9]:
def get_date(date):
    dt = re.findall('[A-Z][^A-Z]*', date)[-1]
    return dt

def fix_n(x):
    x = x.replace('\n', ' ')
    
    if('Full Article' in x):
        x=x.replace('Full Article', ' ')
    return x

In [5]:
url = 'https://technews.acm.org/'
res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
soup = bs4.BeautifulSoup(res.text,'html.parser')

In [6]:
text = soup.findAll('div',{'class':'desktopFontSize'})
text = text[2:]

In [10]:
records = []
for i in range(len(text)):
    Heading = fix_n(text[i].find('b').text)
    Date = get_date(fix_n(text[i].find('i').text))
    Time = '-'
    URL = text[i].findAll('a')[-1]['href']
    Text = fix_n(text[i].text)
    Source = 'ACM'
    Category = 'ACM'
    records.append((Source,Heading,Category,Date,Time,URL,Text))

In [11]:

df=pd.DataFrame(records,columns=['Source','Heading','Category','Date','Time','URL','Text'])
df['Date'] = pd.to_datetime(df['Date'])



In [12]:
df

,Source,Heading,Category,Date,Time,URL,Text
0,ACM,Gymnastics' Latest Twist? AI Judges That See E...,ACM,2019-10-10,-,https://www.nytimes.com/2019/10/10/sports/olym...,Gymnastics' Latest Twist? AI Judges Tha...
1,ACM,Biologically Inspired Artificial Skin Improves...,ACM,2019-10-10,-,https://www.tum.de/nc/en/about-tum/news/press-...,Biologically Inspired Artificial Skin I...
2,ACM,Cybersecurity Giants to Combat Cyberthreats Un...,ACM,2019-10-08,-,https://www.zdnet.com/article/cybersecurity-fi...,Cybersecurity Giants to Combat Cyberthreat...
3,ACM,Doctors Look to Eye-Tracking to Improve Care,ACM,2019-10-10,-,https://www.wsj.com/articles/doctors-look-to-e...,Doctors Look to Eye-Tracking to Improve...
4,ACM,Researchers Use Game Theory to Identify Bacter...,ACM,2019-10-09,-,https://news.wsu.edu/2019/10/09/researchers-us...,Researchers Use Game Theory to Identify...
5,ACM,Drones as Detectives: Surveying Crime Scenes f...,ACM,2019-10-08,-,https://spectrum.ieee.org/tech-talk/robotics/d...,Drones as Detectives: Surveying Crime S...
6,ACM,Engineers Solve 50-Year-Old Puzzle in Signal P...,ACM,2019-10-10,-,https://www.news.iastate.edu/news/2019/10/10/s...,Engineers Solve 50-Year-Old Puzzle in S...
7,ACM,Algorithm Helps Autonomous Vehicles Avoid Pede...,ACM,2019-10-10,-,https://www.theengineer.co.uk/algorithm-av-saf...,Algorithm Helps Autonomous Vehicles Avo...
8,ACM,Flaw in iTunes for Windows Abused for Ransomwa...,ACM,2019-10-10,-,https://www.pcmag.com/news/371261/flaw-in-itun...,Flaw in iTunes for Windows Abused for R...
9,ACM,"Caltech's Latest Creation: A Hovering, Bird-Li...",ACM,2019-10-09,-,https://www.washingtonpost.com/technology/2019...,"Caltech's Latest Creation: A Hovering, Bir..."


In [13]:
ACM = pd.read_csv('ACM_Text')
outname ='ACM_Text'

root = 'Backup/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

ACM.to_csv(fullname,index=False, encoding='utf-8')

In [14]:
s1 = pd.merge(ACM, df, how='inner', on=['Heading'],sort=True)

In [15]:
Headings = s1['Heading']

def fix_heading(x):
    for i in Headings:
        if(i==x):
            return 'nil'
    return x
df['Heading'] = df['Heading'].apply(fix_heading)
df = df[df['Heading']!='nil']
df.head()


,Source,Heading,Category,Date,Time,URL,Text
0,ACM,Gymnastics' Latest Twist? AI Judges That See E...,ACM,2019-10-10,-,https://www.nytimes.com/2019/10/10/sports/olym...,Gymnastics' Latest Twist? AI Judges Tha...
1,ACM,Biologically Inspired Artificial Skin Improves...,ACM,2019-10-10,-,https://www.tum.de/nc/en/about-tum/news/press-...,Biologically Inspired Artificial Skin I...
2,ACM,Cybersecurity Giants to Combat Cyberthreats Un...,ACM,2019-10-08,-,https://www.zdnet.com/article/cybersecurity-fi...,Cybersecurity Giants to Combat Cyberthreat...
3,ACM,Doctors Look to Eye-Tracking to Improve Care,ACM,2019-10-10,-,https://www.wsj.com/articles/doctors-look-to-e...,Doctors Look to Eye-Tracking to Improve...
4,ACM,Researchers Use Game Theory to Identify Bacter...,ACM,2019-10-09,-,https://news.wsu.edu/2019/10/09/researchers-us...,Researchers Use Game Theory to Identify...


In [17]:
temp = ACM.append(df)
temp['Date'] = pd.to_datetime(temp['Date'])
temp = temp.sort_values(by=['Date'],ascending=False).reset_index().drop(['index'],axis=1)
temp=temp[['Source','Heading','Category','Date','Time','URL','Text']]

In [18]:
temp.to_csv('ACM_Text')

In [19]:
outname ='ACM-'+str(Today_date)+'.csv'
root = 'Uncategorized Data/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

In [20]:
df.to_csv(fullname,index=False,encoding='utf-8')